## Семинар 7. Извлечение именованных сущностей.

Именованные сущности - общий термин, который используют для обозначения какого-то множества слов (словосочетаний, последовательностей символов), которые представляют какой-то особый инетерес в контексте решаемой практической задачи и которые нужно отделить от остальных слов. Это могут быть классические имена, фамилии, названия организаций, городов, рек, гор, а могут быть и, например, денежные суммы, даты, статьи кодексов, номера телефонов.


В многих случаях случаях можно написать регуляру и решиьть задачу на приемлимом уровне (например собрать все номера телефонов совсем несложно). Но когда сущности выраются нестандартным образом, постоянно расширяются и меняются по формам, задача становится очень сложной.

К тому же в разных текстах употребляются разные сущности. Поэтому универсальных извлекателей сущностей нет. Есть только стандартные Персоны, Локации, Организации. 

Для английского удобно использовать spacy. Там сразу извлекаются сущности с хорошим качеством.

Для русского (если не хочется ничего делать) можно использовать тэги из pymorphy.

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
p = morph.parse('Михаил')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

In [ ]:
p = morph.parse('Иванов')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

In [ ]:
p = morph.parse('Петрович')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

In [ ]:
p = morph.parse('Москва')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

In [ ]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

In [ ]:
p = morph.parse('')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Работает не очень хорошо, но все равно лучше, чем ничего. Рядом стоящие слова одного тэга можно склеить в один. Или сначала собрать нграмы и если какое-то одно слово в нграмме принадлежит к какому-то типу сущности, то распространить его на весь нграм.

Есть пара библотек, специально предназначенных для этого. Например, natasha - https://github.com/natasha/natasha

Она основана на парсере yargy https://github.com/natasha/yargy и представляет собой набор готовых правил для извлечения некоторых сущностей.

In [ ]:
from natasha import (NamesExtractor,
                     SimpleNamesExtractor,
                     PersonExtractor,
                     LocationExtractor,
                     AddressExtractor,
                     OrganisationExtractor,
                     DatesExtractor,
                     MoneyExtractor,
                     MoneyRateExtractor,
                     MoneyRangeExtractor)

from natasha.markup import (show_markup_notebook as show_markup,
                            format_json)

In [ ]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = NamesExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [ ]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = PersonExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [ ]:
text = 'Более того в Москве, в районе Строгино. На реке Оке. В германии'

extractor_loc = LocationExtractor()
matches = extractor_loc(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [ ]:
text = 'ФСБ. Московский государственный университет. Высшая школа экономика. ВШЭ. Mail.ru'

extractor_org = OrganisationExtractor()
matches = extractor_org(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [ ]:
text = 'С 2015 по 2017 год. 16 апреля 1993 года. В субботу. 23.04.18'

extractor_date = DatesExtractor()
matches = extractor_date(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [ ]:
text = "Он заплатил ему 300 рублей."

extractor_money = MoneyExtractor()
matches = extractor_money(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

В yargy можно писать свои грамматики, подробнее про синтаксис можно почитать в: http://yargy.readthedocs.io/ru/latest/

Ещё есть томита-парсер, но с ним очень тяжело работать (никакого развития, скудная документация, закрытый код, никакого сообщества) https://tech.yandex.ru/tomita/

Для русского state-of-the-art - библеотека от Ipavlov из Физтеха.
https://github.com/deepmipt/ner

Она основана на BiLSTM-CRF (нейронки) и скорее всего просто так не поставится и использовать её будет трудновато (без GPU).

In [ ]:
import ner
extractor = ner.Extractor()

In [ ]:
list(extractor('ФСБ. Московский государственный университет. \
               Высшая школа экономика. ВШЭ. Mail.ru'))

Тэги у них стандартные (PER, LOC, ORG). Если нужно выделять какие-то другие сущности, то нужно размечать свой корпус и обучать свою модель. Даже если нужны только эти категории, может потребоваться новый корпус и модель, так как модели обучены на каких-то общих корпусах (новостных, вики), а на практике приходится работать с какими-то специфичными вещами (рецептами, резюме, судебными решениями).

Размечать корпус дорого и сложно, поэтому люди постоянно пытаются придумать способы сделать NER без разметки корпуса.

Одна из самых старых идей (обычно ссылаются на работу Hearst как первую - http://www.aclweb.org/anthology/C92-2082) это бутстреппинг. Суть в том, чтобы с помощью какого-то точного паттерна извлечь набор сущностей, а потом с помощью этих сущностей собрать новые паттерны (и повторить всё заново).

In [ ]:
import re
from collections import Counter
import string
import numpy as np
from pymorphy2 import MorphAnalyzer

Возьмем небольшой (100 тыс предложений) корпус из википедии и попробуем извлечь какой-то набор организаций.

In [ ]:
candidates = Counter()

# паттерн "компания X+"
# скорее всего, слово с заглавной буквы после слова компания - название компании

pattern = 'компания ([А-ЯЁA-Z«][\w\-\.»]+(?: [А-ЯЁA-Z][\w\-\w\.»]+)*)'
new = Counter()

for line in open('sentences_100k_wiki.txt'):
    candidates.update(re.findall(pattern, line))
      

In [ ]:
candidates.most_common(100)

Таким образом можно извлекать и более узкие вещи.

In [ ]:
candidates = Counter()


pattern = 'прозвищу ([А-ЯЁA-Z][\w-]+(?: [А-ЯЁA-Z][\w-]+)*)'
new = Counter()

for line in open('sentences_100k_wiki.txt'):
    candidates.update(re.findall(pattern, line))
      

In [ ]:
candidates

Можно воспользоваться паттерном - "X и Y", чтобы найти однотипиные слова. Только сначала нужно задать небольшой список примеров.

In [ ]:
candidates = Counter(['Париж', "Берлин", "Москва"])
pattern = '(?:{}) и ([А-ЯЁ][\w-]+)'
new = Counter()
seen = set()

# пройдем по корпусу несколько раз,
# каждый раз добавляя новые примеры
# чтобы не искать по одному и тому же слову,
# сделаем словарь посещенных
# чтобы не включать мусор в поиск, будем искать только по топ-15
for i in range(15):
    print("Итерация - ", i)
    print("Собрано - ", len(candidates))
    cands = set([cand for cand,_ in candidates.most_common(25)])
    cands -= seen
    
    if not cands:
        print('Ничего не найдено!')
        break
    
    for line in open('sentences_100k_wiki.txt'):
        new.update(re.findall(pattern.format('|'.join(cands)), line))
        seen |= cands
        candidates += new
        new = Counter()
      

Получились не совсем города, но под категорию LOC подходит. 

In [ ]:
candidates.most_common(100)

Можно также поискать частотные паттерны, в которых встречается какой-то тип сущности. Используем тэги и pymorphy и достанем 2 предыдущих и два последующих слова.

In [ ]:
morph = MorphAnalyzer()
patterns = Counter()

before_patterns = Counter()

after_patterns = Counter()

for line in open('sentences_100k_wiki.txt'):

    words = ['<START>', ] + line.split() + ['<END>']
    tags = [morph.parse(word.strip(string.punctuation))[0].tag for word in words]
    tags = ['LOC' if 'Geox' in tag else '' for tag in tags]
    inds = []
    start = None
    end = None
    for i, tag in enumerate(tags):
        if tag == 'LOC':
            if start:
                end = i
            else:
                start = i

        else:
            if start:
                end = i
                inds.append((start, end))
                start, end = None, None            

    for ind in inds:
        start, end = ind
        before = max(0, start-2)
        after = end+3
        before_context = ' '.join(words[before:start])
        after_context = ' '.join(words[end+1:after])
        patterns.update([(before_context, after_context)])
        before_patterns.update([before_context])
        after_patterns.update([after_context])

    

In [ ]:
patterns.most_common(10)

Бустреппинг предполагает поочередный поиск патернов и сущностей. Мы этого делать не будет, так как в этом случае появляется проблема с мусорными примерами и паттернами. Оценивание мусорности сущности или паттерна - сложная задача.

Даже единичными паттернами можно набрать какое-то количесто примеров и просто разметить ими корпус. А уже на нем обучить модель, которая захватит всякие контекстуальные признаки и обобщит разметку.

In [ ]:
def label_text(text, gazzeteer, tag):
    labels = []
    text = re.sub('  +', ' ', text)

    for word in gazzeteer:
        start = text.find(word)
        if start >= 0:
            labels.append((start, start+len(word)))
    
    
    words = text.split()
    if not labels:
        return (False, [(word, 'O') for word in words])
    
    spans = []
    i = 0
    for word in words:
        strip_word_right = word.rstrip(string.punctuation)
        strip_word_left = word.lstrip(string.punctuation)

        spans.append((i, i+len(word)-len(strip_word_left), i+len(word), i+len(strip_word_right)))
        i += len(word)
        i += 1

    tags = []
    for span in spans:
        for label in labels:
            if (span[0] >= label[0] or span[1] >= label[0]) \
              and (span[2] <= label[1] or span[3] <= label[1]):
                tags.append(tag)
                break
        else:
            tags.append('O')
    bio_tags = []
    inside = False
    for tag in tags:
        if tag != 'O':
            if inside:
                bio_tags.append(tag+'-I')
            else:
                bio_tags.append(tag+'-B')
                inside = True
        else:
            bio_tags.append(tag)
            inside = False
    
    if any([tag!='O' for tag in bio_tags]):
        return (True, list(zip(words, bio_tags)))
    else:
        return (False, list(zip(words, bio_tags)))
            
    

In [ ]:
orgs = set([org for org, c in candidates.most_common(500)]) - set(['Украины', "ООО", "Гудзонова", "No"])

In [ ]:
text = 'SpaceX представил.'

label_text(text, orgs, 'ORG')

In [ ]:
positive = []
negative = []

for line in open('sentences_100k_wiki.txt'):
    labeled = label_text(line, orgs, 'ORG')
    if labeled[0]:
        positive.append(labeled[1])
    else:
        if np.random.randint(10) > 8:
            negative.append(labeled[1])
        

In [ ]:
# positive = []
# negative = []

# locs = set([loc for loc, c in candidates.most_common(500)])
# for line in open('sentences_100k_wiki.txt'):
#     labeled = label_text(line, locs, 'LOC')
#     if labeled[0]:
#         positive.append(labeled[1])
#     else:
#         if np.random.randint(10) > 8:
#             negative.append(labeled[1])
        

In [ ]:
positive[:100]

Чтобы модель обобщалась обучим fasttext и обучим модель на векторах слов.

In [ ]:
import gensim
import string
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = [[word.strip(string.punctuation) for word in line.lower().split()] for line in open('sentences_100k_wiki.txt')]

In [ ]:
fast_text = gensim.models.FastText(data, max_vocab_size=120000)

In [ ]:
def get_embedding(word, model, dim):
    word = word.lower()
    try:
        v = model[word].reshape(1,-1)
    except (KeyError, ValueError):
        v = np.zeros((dim)).reshape(1,-1)
    
    
    return v

In [ ]:
def get_text_matrix(sents, model, dim):
    size = sum((len(s) for s in sents))
    
    #создадим большую матрицу для всех слов
    X = np.zeros((size, (dim*2)+3))
    y = np.zeros(size, dtype='object')
    
    ind = 0
    # пройдем по всем предложениям
    for i, sent in enumerate(sents):
        # в каждом предложении каждому слову препишем
        # его вектор и вектор предыдушего слова
        # также добавим признак начала предложения
        # и предыдущего тэга
        for j, word in enumerate(sent):
            word, tag = word
            
            if j: # если не начало
                prev_word, prev_tag = sent[j-1]
                if '-B' in prev_tag:
                    prev_tag = [0, 1]
                elif '-I' in prev_tag:
                    prev_tag = [1, 0]
                else:
                    prev_tag = [0, 0]
                prev_vec = get_embedding(prev_word, model, dim)
                start_sent = 0

            else: # если начало
                # нулевой вектор для предыдущего первого слова
                prev_vec = np.zeros((dim)).reshape(1, -1) 
                start_sent = 1
                prev_tag = [0, 0]

            vec =  get_embedding(word, model, dim)

            y[ind] = tag

            X[ind] = np.concatenate([vec, prev_vec, [prev_tag], [[start_sent]]], axis=1)
            
            ind += 1

    return X, y



In [ ]:
X, y = get_text_matrix(positive[:10000], fast_text, 100)

In [ ]:
Counter(y)

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y,random_state=1, stratify=y)
clf = LogisticRegression(C=100, class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

Для предсказания нужно написать отдельную функцию, так как у нас нет изначально нет никакой информации о тэгах. Нужно предсказывать тэг для каждого слова отдельно и передавать его дальше.

In [ ]:
def get_sent_pred(sent, model, dim, clf):
    preds = []
    words = []
    pred_tags = []
    words.append(sent[0])
    vec =  get_embedding(sent[0], model, dim)
    prev_vec = np.zeros((dim)).reshape(1, -1)
    start_sent = 1
    prev_tag = [0, 0]
    v = np.concatenate([vec, prev_vec, [prev_tag], [[start_sent]]], axis=1)
    pred = clf.predict(v)[0]
    pred_tags.append(pred)

    for j, word in enumerate(sent[1:]):

        words.append(word)
        prev_vec = vec
        vec =  get_embedding(word, model, dim)
        start_sent = 0
        
        if '-B' in pred:
            prev_tag = [0, 1]
        elif '-I' in pred:
            prev_tag = [1, 0]
        else:
            prev_tag = [0, 0]
        v = np.concatenate([vec, prev_vec, [prev_tag], [[start_sent]]], axis=1)
        pred = clf.predict(v)[0]

        pred_tags.append(pred)

    
    return list(zip(words, pred_tags))

def get_preds(sents, model, dim, clf):
    
    preds = []

    for i, sent in enumerate(sents):
        pred = get_sent_pred(sent, model, dim, clf)
        preds.append(pred)
    
    return preds

In [ ]:
sents = [[word for word, tag in sent] for sent in negative[10000:20000]]

In [ ]:

preds = get_preds(sents, fast_text, 100, clf)

In [ ]:
[pred for pred in preds if any([x[1] != 'O' for x in pred])]

In [ ]:
negative[:100]

Что-то таким образом предсказывается, но чтобы довести это всё до приличного состояния нужно ещё много улучшений. 

Есть даже фреймворк для такой полуручной разметки данных - https://hazyresearch.github.io/snorkel/ и вообще weak supervision достаточно популярная тема.